In [14]:
work_path = r'../../../'
quetzal_path = work_path + r'quetzal_santo_domingo/'

In [15]:
from tqdm import tqdm

In [16]:
import pandas as pd
parameter_frame = pd.read_csv(quetzal_path + 'inputs/parameters.csv', sep=';', decimal=',').set_index(['category','parameter'])
for c in parameter_frame.columns:
    parent = parameter_frame[c][('general', 'parent')]
    try:
        parent = parent.iloc[0]
    except AttributeError:# 'str' object has no attribute 'iloc'
        pass
    parameter_frame[c] = parameter_frame[c].fillna(parameter_frame[parent])

scenario_list = list(parameter_frame.columns)

In [17]:
pool = {}

In [19]:
df_dict = pd.read_excel(quetzal_path + 'model/ref_18' + '/stacks.xlsx', sheet_name=None)
for key in df_dict.keys():
    pool[key] = []
for scenario in tqdm(scenario_list):
    df_dict = pd.read_excel(quetzal_path + r'model/' + scenario + '/stacks.xlsx', sheet_name=None)
    for key, value in df_dict.items():

        value['scenario'] = scenario
        col = [c for c in value.columns if 'scenario' not in c]
        col.insert(-1, 'scenario')
        value = value[col]
        pool[key].append(value)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  7.27it/s]


In [20]:
stacks = {k: pd.concat(v) for k, v in pool.items()}

In [21]:
with pd.ExcelWriter(quetzal_path + r'outputs/'+ 'stacks.xlsx') as writer:  # doctest: +SKIP
    for name, stack in stacks.items():
        stack.to_excel(writer, sheet_name=name, index=False)